In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import CamembertTokenizer
from math import exp

In [3]:
from tqdm.auto import tqdm # Importation de tqdm pour Colab/Jupyter

def evaluate_model(model, dataloader, device):
    model.eval()
    model.to(device)

    total_loss = 0
    total_masked_tokens = 0
    correct_top1 = 0
    correct_top5 = 0
    criterion = torch.nn.CrossEntropyLoss()

    # Initialisation de la barre de progression
    progress_bar = tqdm(dataloader, desc="Évaluation en cours", leave=True)

    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Inférence
            logits = model(input_ids, attention_mask=attention_mask)

            # Calcul de la Loss
            vocab_size = logits.size(-1)
            loss = criterion(logits.view(-1, vocab_size), labels.view(-1))
            total_loss += loss.item()

            # Mise à jour de la barre avec la loss actuelle
            progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

            # Calcul de la précision sur les tokens masqués
            masked_indices = (labels != -100)
            masked_labels = labels[masked_indices]
            masked_logits = logits[masked_indices]

            if masked_labels.numel() > 0:
                # Top-1
                preds_top1 = masked_logits.argmax(dim=-1)
                correct_top1 += (preds_top1 == masked_labels).sum().item()

                # Top-5
                _, preds_top5 = masked_logits.topk(5, dim=-1)
                correct_top5 += (preds_top5 == masked_labels.unsqueeze(-1)).any(dim=-1).sum().item()

                total_masked_tokens += masked_labels.numel()

    avg_loss = total_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    acc_top1 = (correct_top1 / total_masked_tokens) * 100 if total_masked_tokens > 0 else 0
    acc_top5 = (correct_top5 / total_masked_tokens) * 100 if total_masked_tokens > 0 else 0

    return {
        "Loss": avg_loss,
        "Perplexity": perplexity,
        "Top-1 Acc": acc_top1,
        "Top-5 Acc": acc_top5
    }

def any_in_topk(topk_preds, labels):
    # Utilitaire pour compter les labels présents dans le top K
    return (topk_preds == labels.unsqueeze(-1)).any(dim=-1).sum().item()

def fill_mask_test(model, tokenizer, text, device, top_k=5):
    """
    Évaluation qualitative : prédit les mots les plus probables pour une phrase donnée.
    """
    model.eval()
    model.to(device)

    # Encodage de la phrase avec le token <mask>
    inputs = tokenizer(text, return_tensors="pt").to(device)
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        logits = model(**inputs).logits

    mask_token_logits = logits[0, mask_token_index, :]
    top_k_tokens = torch.topk(mask_token_logits, top_k, dim=-1).indices[0].tolist()

    print(f"\nPhrase : {text}")
    for i, token_id in enumerate(top_k_tokens):
        word = tokenizer.decode([token_id])
        print(f"  {i+1}. {word}")

In [ ]:
import torch
import sys
from transformers import CamembertTokenizer
from cmbrt_lightning_module import CamembertLightning
from cmbrt_data_module import CamembertDataModule  # Correction du nom du fichier/classe

# Nettoyage préventif de la VRAM
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Chargement du Tokenizer et du Modèle
print("Chargement du tokenizer et du modèle...")
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

# Chargement du checkpoint (map_location assure la compatibilité CPU/GPU)
model = CamembertLightning.load_from_checkpoint(
    "/home/camembert/js/totest/camembert-epoch=02-train_loss=3.034.ckpt",
    map_location=device
)
model.eval()

# 3. Évaluation qualitative (Exemples de probing)
# On lit le fichier ligne par ligne pour éviter de saturer la RAM
file_path = "/home/camembert/data/datasets/camemBERT/fr_part_3.txt"
print(f"\nExécution des tests qualitatifs (Probing) à partir de '{file_path}':")

try:
    with open(file_path, "r", encoding="utf-8") as f:
        count = 0
        for line in f:
            sentence = line.strip()
            # On ne teste que les phrases contenant un masque
            if sentence and "<mask>" in sentence:
                fill_mask_test(model, tokenizer, sentence, device)
                count += 1

            # Limite pour ne pas figer la console Colab (ex: 15 premiers tests)
            if count >= 15:
                break
except FileNotFoundError:
    print(f"Erreur: Le fichier '{file_path}' n'a pas été trouvé.")
except Exception as e:
    print(f"Erreur lors de la lecture : {e}")

# 4. Évaluation quantitative (Calcul des métriques)
print("\nPréparation du DataLoader de validation...")

# Le dossier où se trouvent vos fichiers .txt sur votre Drive
drive_base_path = "/home/camembert/data/datasets/camemBERT"

# Initialisation correcte selon la classe fournie
data_module = CamembertDataModule(base_path=drive_base_path, batch_size=8)
# On précise stage="test" pour activer le bypass
data_module.setup(stage="test")
val_dataloader = data_module.val_dataloader()

if val_dataloader:
    print("Démarrage de l'évaluation quantitative...")
    results = evaluate_model(model.model, val_dataloader, device)

    print("\n" + "="*30)
    print("RÉSULTATS QUANTITATIFS")
    print("="*30)
    for metric, value in results.items():
        print(f"{metric:<12} : {value:.4f}")
else:
    print("Erreur : Impossible de charger le DataLoader de validation.")

Chargement du tokenizer et du modèle...
[INFO] CamemBERT config chargée : vocab_size = 32005

Exécution des tests qualitatifs (Probing) à partir de '/home/camembert/data/datasets/camemBERT/fr_part_3.txt':
Erreur: Le fichier '/home/camembert/data/datasets/camemBERT/fr_part_3.txt' n'a pas été trouvé.

Préparation du DataLoader de validation...
[INFO] Chargement des datasets tokenisés depuis le disque 
[INFO] Train : 6,688,051 / Val : 669,953
Démarrage de l'évaluation quantitative...


/home/camembert/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Évaluation en cours:   0%|          | 0/83745 [00:00<?, ?it/s]